In [33]:
import pandas as pd 
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

%matplotlib inline

### Combining Datasets of Different States

In [34]:
data_maharashtra  = pd.read_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Maharashtra-2Years.csv")
data_andhrapradesh = pd.read_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Andhra-2Years.csv")
data_gujarat = pd.read_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Gujarat-2Years.csv")
data_haryana= pd.read_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Haryana-2Years.csv")
combined_data = data_maharashtra.append(data_andhrapradesh).append(data_gujarat).append(data_haryana)

Note: This is Kisan Call Center data for the year 2015 and 2016 of 8 Leading Cottoon Producing States in India

### Analysing the Data

In [35]:
print('Total Number of Queries for the 8 States in the Year 2015 and 2016: ',len(combined_data))

Total Number of Queries for the 8 States in the Year 2015 and 2016:  2520667


In [36]:
print('Number of Queries per State:')
print(combined_data['State'].value_counts())

Number of Queries per State:
MAHARASHTRA       1376489
HARYANA            545790
GUJARAT            331982
ANDHRA PRADESH     266406
Name: State, dtype: int64


In [37]:
crop_counts = pd.DataFrame(data = combined_data.groupby(by=['State','Crop'])['Crop'].count().reset_index(name='count'))
print('Top 5 Crops with Most Queries in these 8 states')
crop_counts.groupby(by=['Crop']).sum().reset_index().sort_values(by='count',ascending = False)[0:5].reset_index()

Top 5 Crops with Most Queries in these 8 states


,index,Crop,count
0,194,Others,1204181
1,79,Cotton (Kapas),207415
2,284,Wheat,108504
3,196,Paddy (Dhan),105908
4,191,Onion,78062


### Cotton Specific Queries

In [38]:
cotton_data = combined_data[combined_data['Crop'] == 'Cotton (Kapas)']
print('Number of Queries for Cotton: {}'.format(len(cotton_data)))
print('Query Type break up within Cotton')
top5_querytype = cotton_data.groupby(by=['QueryType'])['Query'].count().reset_index().sort_values(by='Query',ascending = False).rename(columns={'Query':'Query Type Count'})
top5_querytype['Query Type Count'] = top5_querytype['Query Type Count']/sum(top5_querytype['Query Type Count'])
top5_querytype[0:5]

Number of Queries for Cotton: 207415
Query Type break up within Cotton


,QueryType,Query Type Count
1,\tPlant Protection\t,0.454630
8,Fertilizer Use and Availability,0.104563
10,Market Information,0.095514
21,Weather,0.075612
20,Varieties,0.063674


In [39]:
cotton_data_plant_protection = cotton_data[cotton_data['QueryType'] == '\tPlant Protection\t']
print('Total Number of Queries under Plant Protection Category in Cotton: {}'.format(len(cotton_data_plant_protection)))
print('Top Repeated Queries within Plant Protection in Cotton')
cotton_data_plant_protection['Query'].value_counts()[0:10]

Total Number of Queries under Plant Protection Category in Cotton: 94297
Top Repeated Queries within Plant Protection in Cotton


Ask about sucking pests problem in crop                             2526
Attack of White Fly?                                                2348
Attack of Thrips and Jassids?                                       2261
information regardingh how to control fangle disease in cotton ?    2149
how to control leaf carl in cotton crops?                           1995
ATTACK OF SUCKING PEST?                                             1530
Ask about heliothese (pink ball warm) in cotton                     1295
Attack of Thrips?                                                   1257
How to control white fly,trips in cotton?                           1234
Ask about larvae control information                                1223
Name: Query, dtype: int64


Keeping only unique query-answer pair per District/State/Block

In [40]:
cotton_data_plant_protection_dedup = cotton_data_plant_protection.drop_duplicates(subset=['Query','State','District','Block'],keep ='first')

In [41]:
print('Number of Unique Queries per State, District, Block : {}'.format(len(cotton_data_plant_protection_dedup)))

Number of Unique Queries per State, District, Block : 52908


##### Removing Location Specific Querying for Now and keeping one query answer pair for the entire data of 8 states

In [42]:
cotton_data_plant_protection_dedup1 = cotton_data_plant_protection.drop_duplicates(subset=['Query'],keep ='first')
cotton_data_plant_protection_dedup1['Query'] = cotton_data_plant_protection_dedup1["Query"].map(lambda x: x.lower());
print('Number of Unique Queries for all the cotton queries of the 8 States  : {}'.format(len(cotton_data_plant_protection_dedup1)))
cotton_data_plant_protection_dedup2 = cotton_data_plant_protection_dedup1.dropna()
print('Number of Unique Queries for all the cotton queries of the 8 States (After Removing Blank Queries) : {}'.format(len(cotton_data_plant_protection_dedup2)))

Number of Unique Queries for all the cotton queries of the 8 States  : 12693
Number of Unique Queries for all the cotton queries of the 8 States (After Removing Blank Queries) : 7099


C:\Users\Manan Arora\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [102]:
cotton_data_plant_protection_dedup2.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Cotton-QnA-Raw_v1.csv',index = False)

In [45]:
#Question Type 1: ATTACK
cotton_data_plant_protection_attack = cotton_data_plant_protection_dedup2[cotton_data_plant_protection_dedup2['Query'].str.contains("attack")].reset_index()
print('Number of Questions that talked about Attack on Cotton Plants: {}'.format(len(cotton_data_plant_protection_attack)))
# Question Type 2: ASK
cotton_data_plant_protection_ask = cotton_data_plant_protection_dedup2[cotton_data_plant_protection_dedup2['Query'].str.contains("ask")].reset_index()
print('Number of Questions that asked something about the Cotton Plants: {}'.format(len(cotton_data_plant_protection_ask)))
# Question Type 2: Control
cotton_data_plant_protection_control = cotton_data_plant_protection_dedup2[cotton_data_plant_protection_dedup2['Query'].str.contains("control")].reset_index()
print('Number of Questions that asked about controling something in the Cotton Plants: {}'.format(len(cotton_data_plant_protection_control)))

Number of Questions that talked about Attack on Cotton Plants: 1883
Number of Questions that asked something about the Cotton Plants: 2217
Number of Questions that asked about controling something in the Cotton Plants: 1361


### Processing for Chatbot

In [109]:
## Cleaner Function: Used to clean the Queries so that we can have entities to be used in Chatbot
def cleaner(review):
    '''
    Clean and preprocess each query

    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''

    review = bs.BeautifulSoup(review).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    
    #3. Remove punctuation
    review = re.sub("[^a-zA-Z]", " ",review)
    
    review = review.lower().split()

    
    #2. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    review1 = [w for w in review if not w in eng_stopwords]
    
    other_words = set(['cotton','attack','disease','crop','plant','ask','asked','ask','about','control','coton','measure','information','regarding','weather','forecast','asking','information','informationask','farmer', 'want','know','information'])
    review2 = [w for w in review1 if not w in other_words]

    
    #6. Join the review to one sentence
    review = ' '.join(review2)
    # add emoticons to the end

    return(review)

Processing Questions on Attack

In [60]:
query_len = len(cotton_data_plant_protection_attack['Query'])

query_attack_clean = []

for i in range(0,query_len):
    if( (i+1)%100 == 0):
        print("Done with %d queries" %(i+1)) 
    query_attack_clean.append(cleaner(cotton_data_plant_protection_attack['Query'][i]))
cotton_data_plant_protection_attack['Ouery_Cleaned'] = query_attack_clean

Done with 100 queries
Done with 200 queries
Done with 300 queries
Done with 400 queries
Done with 500 queries
Done with 600 queries
Done with 700 queries
Done with 800 queries
Done with 900 queries
Done with 1000 queries
Done with 1100 queries
Done with 1200 queries
Done with 1300 queries
Done with 1400 queries
Done with 1500 queries
Done with 1600 queries
Done with 1700 queries
Done with 1800 queries


In [61]:
cotton_data_plant_protection_attack_dedup = cotton_data_plant_protection_attack[['Ouery_Cleaned','Answer']].drop_duplicates(subset=['Ouery_Cleaned'],keep='first')
print(len(cotton_data_plant_protection_attack_dedup))

532


In [64]:
cotton_data_plant_protection_attack_dedup.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Cotton-Attack-QnA-Processes_v1.csv',index = False)

Processesing Questions like Ask About

In [74]:
query_len = len(cotton_data_plant_protection_ask['Query'])

query_ask_clean = []

for i in range(0,query_len):
    if( (i+1)%100 == 0):
        print("Done with %d queries" %(i+1)) 
    query_ask_clean.append(cleaner(cotton_data_plant_protection_ask['Query'][i]))
cotton_data_plant_protection_ask['Ouery_Cleaned'] = query_ask_clean
cotton_data_plant_protection_ask_dedup = cotton_data_plant_protection_ask[['Ouery_Cleaned','Answer']].drop_duplicates(subset=['Ouery_Cleaned'],keep='first')
print(len(cotton_data_plant_protection_ask_dedup))

Done with 100 queries
Done with 200 queries
Done with 300 queries
Done with 400 queries
Done with 500 queries
Done with 600 queries
Done with 700 queries
Done with 800 queries
Done with 900 queries
Done with 1000 queries
Done with 1100 queries
Done with 1200 queries
Done with 1300 queries
Done with 1400 queries
Done with 1500 queries
Done with 1600 queries
Done with 1700 queries
Done with 1800 queries
Done with 1900 queries
Done with 2000 queries
Done with 2100 queries
Done with 2200 queries
962


In [76]:
cotton_data_plant_protection_ask_dedup.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Cotton-Ask-QnA-Processes_v1.csv',index = False)

Processing Questions about Control

In [84]:
query_len = len(cotton_data_plant_protection_control['Query'])

query_control_clean = []

for i in range(0,query_len):
    if( (i+1)%100 == 0):
        print("Done with %d queries" %(i+1)) 
    query_control_clean.append(cleaner(cotton_data_plant_protection_control['Query'][i]))
cotton_data_plant_protection_control['Ouery_Cleaned'] = query_control_clean
cotton_data_plant_protection_control_dedup = cotton_data_plant_protection_control[['Ouery_Cleaned','Answer']].drop_duplicates(subset=['Ouery_Cleaned'],keep='first')
print(len(cotton_data_plant_protection_control_dedup))

Done with 100 queries
Done with 200 queries
Done with 300 queries
Done with 400 queries
Done with 500 queries
Done with 600 queries
Done with 700 queries
Done with 800 queries
Done with 900 queries
Done with 1000 queries
Done with 1100 queries
Done with 1200 queries
Done with 1300 queries
709


In [86]:
cotton_data_plant_protection_control_dedup.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Cotton-Control-QnA-Processes_v1.csv',index = False)

### Wheat Specific Queries

In [87]:
wheat_data = combined_data[combined_data['Crop'] == 'Wheat']
print('Number of Queries for Wheat: {}'.format(len(wheat_data)))
print('Query Type break up within Wheat')
top5_querytype = wheat_data.groupby(by=['QueryType'])['Query'].count().reset_index().sort_values(by='Query',ascending = False).rename(columns={'Query':'Query Type Count'})
top5_querytype['Query Type Count'] = top5_querytype['Query Type Count']/sum(top5_querytype['Query Type Count'])
top5_querytype[0:5]

Number of Queries for Wheat: 108504
Query Type break up within Wheat


,QueryType,Query Type Count
21,Weather,0.248839
1,\tPlant Protection\t,0.248645
20,Varieties,0.102300
4,Bio-Pesticides and Bio-Fertilizers,0.090034
8,Fertilizer Use and Availability,0.054671


In [88]:
wheat_data_plant_protection = wheat_data[wheat_data['QueryType'] == '\tPlant Protection\t']
print('Total Number of Queries under Plant Protection Category in wheat: {}'.format(len(wheat_data_plant_protection)))
print('Top Repeated Queries within Plant Protection in Wheat')
wheat_data_plant_protection['Query'].value_counts()[0:10]

Total Number of Queries under Plant Protection Category in wheat: 26979
Top Repeated Queries within Plant Protection in Wheat


Information regarding control of zinc deficiency in wheat ?         1986
Information regarding how to control aphids/sundi in wheat crop?    1288
Information regarding how to control mixed weeds in wheat crop ?    1172
How to control yellow rust in wheat crop?                           1109
information regarding how to control yellow rust in wheat crop?      929
information regarding control of affids and thrips in wheat?         542
How to control fungal desease in wheat crop ?                        533
information regarding how to control of termite in wheat?            496
how to control aphids in wheat crop?                                 366
Information regarding how to control yellow rust in wheat crop?      324
Name: Query, dtype: int64


Keeping only unique query-answer pair per District/State/Block

In [91]:
wheat_data_plant_protection_dedup = wheat_data_plant_protection.drop_duplicates(subset=['Query','State','District','Block'],keep ='first')

In [92]:
print('Number of Unique Queries per State, District, Block : {}'.format(len(wheat_data_plant_protection_dedup)))

Number of Unique Queries per State, District, Block : 15082


##### Removing Location Specific Querying for Now and keeping one query answer pair for the entire data of 8 states

In [93]:
wheat_data_plant_protection_dedup1 = wheat_data_plant_protection.drop_duplicates(subset=['Query'],keep ='first')
wheat_data_plant_protection_dedup1['Query'] = wheat_data_plant_protection_dedup1["Query"].map(lambda x: x.lower());
print('Number of Unique Queries for all the wheat queries of the 8 States  : {}'.format(len(wheat_data_plant_protection_dedup1)))
wheat_data_plant_protection_dedup2 = wheat_data_plant_protection_dedup1.dropna()
print('Number of Unique Queries for all the wheat queries of the 8 States (After Removing Blank Queries) : {}'.format(len(wheat_data_plant_protection_dedup2)))

Number of Unique Queries for all the wheat queries of the 8 States  : 3910
Number of Unique Queries for all the wheat queries of the 8 States (After Removing Blank Queries) : 3091


C:\Users\Manan Arora\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [103]:
wheat_data_plant_protection_dedup2.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Wheat-QnA-Raw_v1.csv',index = False)

In [106]:
#Question Type 1: ATTACK
wheat_data_plant_protection_information = wheat_data_plant_protection_dedup2[wheat_data_plant_protection_dedup2['Query'].str.contains("information")].reset_index()
print('Number of Questions that talked about providing information on Cotton Plants: {}'.format(len(wheat_data_plant_protection_attack)))
# Question Type 2: ASK
wheat_data_plant_protection_ask = wheat_data_plant_protection_dedup2[wheat_data_plant_protection_dedup2['Query'].str.contains("ask")].reset_index()
print('Number of Questions that asked something about the Cotton Plants: {}'.format(len(wheat_data_plant_protection_ask)))
# Question Type 2: Control
wheat_data_plant_protection_control = wheat_data_plant_protection_dedup2[wheat_data_plant_protection_dedup2['Query'].str.contains("control")].reset_index()
print('Number of Questions that asked about controling something in the Cotton Plants: {}'.format(len(wheat_data_plant_protection_control)))

Number of Questions that talked about providing information on Cotton Plants: 1235
Number of Questions that asked something about the Cotton Plants: 348
Number of Questions that asked about controling something in the Cotton Plants: 1222


In [110]:
query_len = len(wheat_data_plant_protection_information['Query'])

query_info_clean = []

for i in range(0,query_len):
    if( (i+1)%100 == 0):
        print("Done with %d queries" %(i+1)) 
    query_info_clean.append(cleaner(wheat_data_plant_protection_information['Query'][i]))
wheat_data_plant_protection_information['Ouery_Cleaned'] = query_info_clean

Done with 100 queries
Done with 200 queries
Done with 300 queries
Done with 400 queries
Done with 500 queries
Done with 600 queries
Done with 700 queries
Done with 800 queries
Done with 900 queries
Done with 1000 queries
Done with 1100 queries
Done with 1200 queries


In [116]:
wheat_data_plant_protection_information_dedup = wheat_data_plant_protection_information[['Ouery_Cleaned','Answer']].drop_duplicates(subset=['Ouery_Cleaned'],keep='first')
print(len(wheat_data_plant_protection_information_dedup))
wheat_data_plant_protection_information.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Wheat-Information-QnA-Processes_v1.csv',index = False)

614


In [118]:
query_len = len(wheat_data_plant_protection_ask['Query'])

query_ask_clean = []

for i in range(0,query_len):
    if( (i+1)%100 == 0):
        print("Done with %d queries" %(i+1)) 
    query_ask_clean.append(cleaner(wheat_data_plant_protection_ask['Query'][i]))
wheat_data_plant_protection_ask['Ouery_Cleaned'] = query_ask_clean

Done with 100 queries
Done with 200 queries
Done with 300 queries


In [120]:
wheat_data_plant_protection_ask = wheat_data_plant_protection_ask[['Ouery_Cleaned','Answer']].drop_duplicates(subset=['Ouery_Cleaned'],keep='first')
print(len(wheat_data_plant_protection_ask))
wheat_data_plant_protection_ask.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Wheat-Ask-QnA-Processes_v1.csv',index = False)

236


In [121]:
query_len = len(wheat_data_plant_protection_control['Query'])

query_control_clean = []

for i in range(0,query_len):
    if( (i+1)%100 == 0):
        print("Done with %d queries" %(i+1)) 
    query_control_clean.append(cleaner(wheat_data_plant_protection_control['Query'][i]))
wheat_data_plant_protection_control['Ouery_Cleaned'] = query_control_clean

Done with 100 queries
Done with 200 queries
Done with 300 queries
Done with 400 queries
Done with 500 queries
Done with 600 queries
Done with 700 queries
Done with 800 queries
Done with 900 queries
Done with 1000 queries
Done with 1100 queries
Done with 1200 queries


In [122]:
wheat_data_plant_protection_control = wheat_data_plant_protection_control[['Ouery_Cleaned','Answer']].drop_duplicates(subset=['Ouery_Cleaned'],keep='first')
print(len(wheat_data_plant_protection_control))
wheat_data_plant_protection_control.to_csv(r'C:\Users\Manan Arora\Desktop\Plaksha TLF\Term 2\DataX Project\Agriculture\Datatsets\KCC-Wheat-Control-QnA-Processes_v1.csv',index = False)

644
